In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers;
!pip install sentencepiece;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.2 MB/s eta 0:00:00


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
%cd /content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/lib/
!pwd

/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/lib
/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/lib


In [5]:
from tokenization_kobert import KoBertTokenizer

import json
from copy import deepcopy
import random
from tqdm.notebook import tqdm

from transformers import (
    PreTrainedTokenizerFast as BaseGPT2Tokenizer,
    EncoderDecoderModel,
)



In [6]:
src_tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [7]:
class GPT2Tokenizer(BaseGPT2Tokenizer):
    def build_inputs_with_special_tokens(self, token_ids, _):
        return token_ids + [self.eos_token_id]
trg_tokenizer = GPT2Tokenizer.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')
add_tokenizer = GPT2Tokenizer.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

## 토크나이저 테스팅

In [9]:
new_words = ["씡" , "췬씡" , "궿"]
add_tokenizer.add_tokens(new_words)

3

In [10]:
tstr = "씡췬 췬궿 췬장 궿췬씡"

In [13]:
print(trg_tokenizer.tokenize(tstr))

['▁', '<unk>', '췬', '▁', '췬', '<unk>', '▁', '췬', '장', '▁', '<unk>', '췬', '<unk>']


In [14]:
trg_tokenizer.encode(tstr)

[739, 5, 8409, 739, 8409, 5, 739, 8409, 8168, 739, 5, 8409, 5]

In [15]:
print(trg_tokenizer.decode(trg_tokenizer.encode(tstr)))
print(add_tokenizer.decode(add_tokenizer.encode(tstr)))
print((trg_tokenizer.encode(tstr)))
print((add_tokenizer.encode(tstr)))

<unk>췬 췬<unk> 췬장 <unk>췬<unk>
씡 췬 췬궿 췬장 궿췬씡
[739, 5, 8409, 739, 8409, 5, 739, 8409, 8168, 739, 5, 8409, 5]
[51200, 739, 8409, 739, 8409, 51202, 739, 8409, 8168, 739, 51202, 51201]


In [16]:
print(trg_tokenizer.decode(trg_tokenizer.encode(tstr)))
print(add_tokenizer.decode(add_tokenizer.encode(tstr)))
print((trg_tokenizer.encode(tstr)))
print((add_tokenizer.encode(tstr)))

<unk>췬 췬<unk> 췬장 <unk>췬<unk>
씡 췬 췬궿 췬장 궿췬씡
[739, 5, 8409, 739, 8409, 5, 739, 8409, 8168, 739, 5, 8409, 5]
[51200, 739, 8409, 739, 8409, 51202, 739, 8409, 8168, 739, 51202, 51201]


In [22]:
print(add_tokenizer.decode([5]))

<unk>


## 모델 불러오기

In [8]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained(
    'monologg/kobert',
    'skt/kogpt2-base-v2',
    pad_token_id=trg_tokenizer.bos_token_id
)
model.config.decoder_start_token_id = trg_tokenizer.bos_token_id
model

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['transformer.h.11.crossattention.q_attn.weight', 'transformer.h.10.crossattention.bias', 'transformer.h.2.crossattention.masked_bias', 'transformer.h.1.crossattention.c_proj.bias', 'transformer.h.6.ln_cross_attn.weight', 'transformer.h.9.crossattention.c_attn.weight', 'transformer.h.2.crossattention.c_proj.weight', 'transformer.h.10.crossattention.q_attn.weight', 'transformer.h.3.ln_cross_attn.weight', 'transformer.h.5.crossattention.c_proj.bias', 'transformer.h.2.crossattention.c_attn.weight', 'transformer.h.9.crossattention.c_proj.bias', 'transformer.h.10.crossattention.masked_bias', 'transformer.h.6.crossattention.c_proj.weight', 'transformer.h.9.crossattention.masked_bias', 'transformer.h.8.crossattention.bias', 'transformer.h.3.crossattention.c_proj.bias', 'transformer.h.4.crossattention.bias', 'transformer.h.5.crossattention.bias', 'transformer.h.5.cros

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

## 토크나이저 단어추가 및 모델 임베딩 변경.

In [9]:
!pip install konlpy

from konlpy.tag import Okt

okt = Okt()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 19.7 MB/s eta 0:00:00


In [10]:
with open("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/no_commit/real_word_data.json", 'rb') as f:
    dw_dict = json.load(f)
new_words_d = []
new_words = []
cnt_words = []
for cnt in range(4,9):
    new_words = []
    for i,v in tqdm(enumerate(dw_dict["utterance"])):
        if 5 in trg_tokenizer.encode(v["dialect_form"]) and len(trg_tokenizer.encode(v["dialect_form"])) == cnt and not (v["dialect_form"] in cnt_words):
            morphs = okt.morphs(v["dialect_form"])
            for m in morphs:
                if not (m in new_words_d):
                    new_words.append(m)
                    new_words_d.append(m)
            cnt_words.append(v["dialect_form"])
            print(trg_tokenizer.decode(trg_tokenizer.encode(v["dialect_form"])), v["dialect_form"], len(trg_tokenizer.encode(v["dialect_form"])))

# 추가할 단어를 토큰화합니다

    # 토크나이저에도 새로운 단어를 추가합니다
    trg_tokenizer.add_tokens(new_words)


0it [00:00, ?it/s]

<unk><unk>한 똣똣한 4
어<unk>져게 어딨져게 4
지나실<unk>디 지나실껀디 4
버려실<unk>디 버려실껀디 4
해<unk>수다 해줨수다 4
<unk>게만은 씅게만은 4
<unk>디강 헙디강 4
관<unk>신가 관둼신가 4
<unk>디다 헙디다 4
<unk>으내 봥으내 4
데<unk>내둔 데꼉내둔 4
고르랜<unk>써 고르랜헙써 4
<unk>시냐 봠시냐 4
어<unk>수과 어딨수과 4
냅<unk>하게 냅쌱하게 4
<unk>서예 쏩서예 4
<unk>이네 줭이네 4
바<unk>마씸 바꿘마씸 4
시<unk>이네 시켱이네 4
<unk>이네 놩이네 4
해<unk>이네 해봥이네 4
<unk>신가 꼄신가 4
<unk>마씸 놘마씸 4
또<unk>할때 또똣할때 4
느<unk>신지는 느꼄신지는 4
<unk>을줄 덖을줄 4
<unk>으는 덖으는 4
알려<unk>마심 알려줜마심 4
<unk>아신고 촞아신고 4
도와<unk>꾸나 도와줨꾸나 4
주서봐<unk> 주서봐봔 4
세<unk>그냉 세웡그냉 4
<unk>수다 됫수다 4
<unk>니다 헙니다 4
디밀려<unk> 디밀려뒁 4
수술해여<unk> 수술해여뒁 4
알람맞<unk> 알람맞춴 4
안<unk>주게 안줫주게 4
채점<unk>대 채점됫대 4
<unk>아주마 촞아주마 4
<unk>기엔 촞기엔 4
<unk>아완 촞아완 4
<unk>아왕 촞아왕 4
끄<unk>이엔 끄껭이엔 4
<unk>아낸 촞아낸 4
<unk>째번 츳째번 4
비<unk>먹젠 비웡먹젠 4
<unk>허는 옝허는 4
배와<unk>고 배와젔고 4
착신시<unk> 착신시켱 4
<unk><unk>하게 똣똣하게 4
족을<unk>가 족을껀가 4
파<unk>으네 파놩으네 4
<unk>었쥬 걲었쥬 4
재<unk>으네 재웡으네 4
내려<unk>으네 내려놩으네 4
<unk>으네 놩으네 4
정해<unk>으네 정해놩으네 4
<unk>주게 됬주게 4
크지않아<unk> 크지않아놘 4
안<unk>주게 안됬주게 4
익을<unk>디 익을껀디 4
<unk>수과 놤수과 4
잘<unk>신게 잘됬신

0it [00:00, ?it/s]

받아주<unk>는디 받아주켔는디 5
<unk>니다게 멉니다게 5
잡아 질<unk>디다 게 잡아질땝디다게 5
안<unk>던거라 안줫던거라 5
<unk>어놔서 틑어놔서 5
<unk>단주난 곴단주난 5
버<unk>시꺼라 버텸시꺼라 5
<unk>기는거라 맫기는거라 5
<unk>앙온거 똗앙온거 5
섭<unk>한거라 섭췰한거라 5
맙<unk>핸야 맙쎙핸야 5
놔<unk>저게 놔둿저게 5
<unk>앙이넹 쌂앙이넹 5
놔<unk>서라 놔덨서라 5
<unk>앙으네 욖앙으네 5
<unk>니께게 뭡니께게 5
<unk>으는디 똗으는디 5
<unk>직<unk>직하게 삔직삔직하게 5
<unk>찍<unk>찍 삔찍삔찍 5
<unk>자리해온건 묏자리해온건 5
<unk>작이라는디 띵작이라는디 5
<unk>부기 낭 쑴부기 낭 5
<unk>비기낭 쑴비기낭 5
만들어 <unk>쥬마씸 만들어봣쥬마씸 5
<unk>다그넹 켔다그넹 5
상담해<unk>쥬게 상담해봣쥬게 5
<unk>었으니까 껶었으니까 5
<unk>거불지 덛거불지 5
괜<unk>아젼 괜칞아젼 5
고라 <unk>는디 고라쌋는디 5
솔<unk>카부댄 솔찟카부댄 5
<unk>겨수다 게 맫겨수다게 5
괜찮아<unk> 괜찮아긇 5
뽀슬<unk>수다 뽀슬뢌수다 5
<unk>는거라 궏는거라 5
<unk>착거리멍 늩착거리멍 5
<unk>껴지지도 벧껴지지도 5
<unk>어그넹 덛어그넹 5
<unk><unk>대고이 켁켁대고이 5
<unk>이라부난 쉣이라부난 5
<unk>와불믄 쒸와불믄 5
<unk>게만은도 눵게만은도 5
할거<unk>허멍 할거넁허멍 5
육<unk> 시아지방 육뚠 시아지방 5
<unk>놨수게 쒜놨수게 5
<unk>으다만은 헜으다만은 5
놔<unk>가지고 놔눵가지고 5
<unk>암서라 촟암서라 5
맞<unk>집니까게 맞촤집니까게 5
잡<unk>댄마씸 게 잡솼댄마씸게 5
<unk>니다게 뗍니다게 5
<unk>아먹언 썲아먹언 5
나와버<unk>쥬게 나와버맀쥬게 5
<unk><unk>하멍 큭큭하멍 5
먹어 짐<unk>커다 먹어짐짔커다 5
대꼉 나<un

0it [00:00, ?it/s]

화해시 <unk>수게 화해시켯수게 6
양양 안 <unk>수다 게 양양 안 쒔수다게 6
폴고허난 게식<unk> 폴고허난게식혠 6
맨도롱<unk>헌때 맨도롱똣헌때 6
<unk>아져불고 똟아져불고 6
<unk>거우다게 쎌거우다게 6
씌왕심<unk>왔주게 씌왕심궝왔주게 6
고라 <unk>대난게 고라짔대난게 6
<unk>어와나서덜 끋어와나서덜 6
<unk>겅낭으네 덛겅낭으네 6
<unk>피어났져 궘피어났져 6
<unk>댄마씀게 꼇댄마씀게 6
<unk>지근허다 촙지근허다 6
<unk>지근헌디 촙지근헌디 6
<unk>지근헌것 촙지근헌것 6
<unk>아본거라 삮아본거라 6
<unk>라됭으네 꽐라됭으네 6
질어<unk>주마 씸 질어욌주마씸 6
옵<unk>해그넹 옵쎙해그넹 6
<unk>어실테쥬 틑어실테쥬 6
매<unk>다그넹에 매꼇다그넹에 6
잘살아 졈<unk>들 잘살아졈쩬들 6
<unk>어먹어 부난 틑어먹어부난 6
<unk>여졌주기게 셖여졌주기게 6
헤켁켁 <unk>해넹 헤켁켁켁해넹 6
<unk><unk>허덴마씨 궙궙허덴마씨 6
<unk><unk>거리는게만 삑삑거리는게만 6
뭉<unk>그려 그넹예 뭉틍그려그넹예 6


0it [00:00, ?it/s]

<unk>쳐져불면예 곂쳐져불면예 7
초라하여져<unk>영이네 초라하여져븨영이네 7
깰끔<unk>금거리멍 깰끔꺨금거리멍 7
<unk>게보락한거 똡게보락한거 7
<unk>어났주마 씸 꺠어났주마씸 7


0it [00:00, ?it/s]

배나왐쥬해<unk>만은이 배나왐쥬해쪔만은이 8
괜<unk>은거닮은디 괜챦은거닮은디 8


In [11]:
# # 토크나이저에도 새로운 단어를 추가합니다
# src_tokenizer.add_tokens(new_words)

# 인코더의 임베딩 크기를 조정합니다
model.decoder.resize_token_embeddings(len(trg_tokenizer))

# 추가한 단어가 토크나이저에 적용되었는지 확인합니다
print(trg_tokenizer.decode(trg_tokenizer.encode(new_words[0])))

배


In [12]:
%cd /content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/standard_to_dialect/no_commit

/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/standard_to_dialect/no_commit


In [13]:
# # 토크나이저들을 모델에 저장합니다
model.config.encoder_attention_type = 'full'
model.config.decoder_attention_type = 'full'
model.config.src_vocab_size = len(src_tokenizer)
model.config.tgt_vocab_size = len(trg_tokenizer)

# push_to_hub() 함수를 사용하여 모델을 Hugging Face Hub에 업로드합니다
model.push_to_hub("ossp-reverse-v0_2")


Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/leadawon/ossp-reverse-v0_2/commit/5911d72c8940e88ed80c11c02d1a2e6d56e84578', commit_message='Upload model', commit_description='', oid='5911d72c8940e88ed80c11c02d1a2e6d56e84578', pr_url=None, pr_revision=None, pr_num=None)

In [14]:
import pickle
# with open('save_edm.pickle', 'wb') as f:
#     pickle.dump(model, f)
# with open('save_src_tk.pickle', 'wb') as f:
#     pickle.dump(src_tokenizer, f)
with open('save_trg_tk.pickle', 'wb') as f:
    pickle.dump(trg_tokenizer, f)

In [16]:
with open('save_trg_tk.pickle','rb') as f:
    add_tokenizer=pickle.load(f)
    

In [ ]:
%cd ..

In [17]:
print(len(add_tokenizer))

54000


## 형태소 분석기 테스트

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#install mecab
from konlpy.tag import Kkma, Komoran, Okt, Mecab

#mec = Mecab()
okt = Okt()
kkm = Kkma()
kom = Komoran()

In [ ]:
with open("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/no_commit/real_word_data.json", 'rb') as f:
    dw_dict = json.load(f)
cnt_words = []

import time
for i,v in tqdm(enumerate(dw_dict["utterance"])):
    if 0 in src_tokenizer.encode(v["dialect_form"]) and not (v["dialect_form"] in cnt_words):
        cnt_words.append(v["dialect_form"])
        print(v["dialect_form"])
        print(okt.morphs(v["dialect_form"]))
        print("#"*50)

0it [00:00, ?it/s]

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
차가와수꽈
['차가와', '수', '꽈']
['▁차', '가', '와', '수', '꽈']
##################################################
무사마씸게
['무사마씸', '게']
['▁무', '사', '마', '씸', '게']
##################################################
겅해부나네
['겅', '해부', '나네']
['▁', '겅', '해', '부', '나', '네']
##################################################
이추룩하믄
['이', '추', '룩', '하믄']
['▁이', '추', '룩', '하', '믄']
##################################################
쭐면
['쭐', '면']
['▁', '쭐', '면']
##################################################
쭐었어이
['쭐었', '어', '이']
['▁', '쭐', '었', '어', '이']
##################################################
냄쪄
['냄', '쪄']
['▁', '냄', '쪄']
##################################################
없수꽈
['없수꽈']
['▁없', '수', '꽈']
##################################################
돌앙댕겸주
['돌', '앙', '댕겸주']
['▁돌', '앙', '댕', '겸', '주']
##################################################
물엉댕기잖아
['물', '엉', '댕기', '잖아']
['▁물', '엉', '댕', '기', '잖', '아']
###############################################

KeyboardInterrupt: ignored

In [ ]:
print(len(src_tokenizer))
print(len(add_tokenizer))

12291
8002


In [ ]:
# with open("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/no_commit/real_word_data.json", 'rb') as f:
#     dw_dict = json.load(f)
# cnt_words = []

# ddinga = ["메시근하다","납시근하다","가멘","먹엇수과"]
# ddingb = ["메시근+하다","납+시근하다","가+ㅁ+엔","먹+엇+수+과"]
# # 미지근하다, 납작스름하다. 가다. 먹었습니까?
# import time
# for i,v in tqdm(enumerate(ddinga)):
#         print(ddinga[i])
#         print(okt.morphs(ddinga[i]))
#         print(ddingb[i])
#         print("#"*50)

0it [00:00, ?it/s]

메시근하다
['메', '시근하다']
메시근+하다
##################################################
납시근하다
['납', '시근하다']
납+시근하다
##################################################
가멘
['가', '멘']
가+ㅁ+엔
##################################################
먹엇수과
['먹엇수', '과']
먹+엇+수+과
##################################################
